In [2]:
# !pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 9.6 MB/s eta 0:00:00:00:0100:01



#### Infos sobre palavras
*Palavras já vistas*
- Quantas vezes você viu essa palavra
- Nivel de conhecimento sobre a palavra
    - Se ja viu 100 vezes e das 100 ccertou 80% você domina essa palavra | se 10% você tem muita dificuldade nessa palavra


*Palavras não vistas*

---

#### Jogos
- jogo da forca

- Inclusões
    - Escrever, escultar e adivinhar
    - Falar a palavra/frase



---

#### Requisitos
- Ao ver uma palavra ela já dev

In [97]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import os
import json
import pygame
from pathlib import Path

pygame.mixer.init()

DATA_PATH = Path("data_organize")

class DataLoader:
    def __init__(self, base_path=DATA_PATH):
        self.base_path = base_path

    def get_categories(self):
        return [self.base_path / path for path in os.listdir(self.base_path)]

    def get_subcategories(self, category_path):
        return [category_path / path for path in os.listdir(category_path)]

    def get_words(self, subcategory_path):
        return [subcategory_path / path for path in os.listdir(subcategory_path)]

    def get_all_words(self, subcategory_name=None):
        palavras = []
        for categoria in self.get_categories():
            for subcat in self.get_subcategories(categoria):
                if subcategory_name and subcat.name != subcategory_name:
                    continue
                palavras.extend(self.get_words(subcat))
        return palavras


class Palavra:
    def __init__(self, path):
        self.path = path
        self.arquivos = list(os.listdir(path))
        self.texto_info = self._carregar_texto()
        self.imagens = self._carregar_imagens()


        self.text_pt_br = self.texto_info.get("recorte_1.jpg", None)
        self.text_eng = self.texto_info.get("recorte_3.jpg", None)
        self.audio_path = path / "audio.wav"
        self.fonetica_path = path / "recorte_4.jpg"

    def _carregar_texto(self):
        try:
            with open(self.path / "text.json", "r", encoding="utf-8") as f:
                return json.load(f)
        except:
            return {}

    def _carregar_imagens(self):
        return sorted(
            [self.path / arq for arq in self.arquivos if arq.startswith("recorte_") and arq.endswith(".jpg") and "recorte_4" not in arq],
            key=lambda x: int(x.stem.split("_")[1])
        )

loader = DataLoader()
palavras_paths = loader.get_all_words(subcategory_name="descrição_da_casa")

# self.palavra = Palavra(self.palavras_paths[self.indice])
# self.progresso_label.config(text=f"{self.indice + 1}/{len(self.palavras_paths)}")

palavra = Palavra(palavras_paths.pop())

palavra.text_eng


'clean'

'clean'

In [98]:


class AudioPlayer:
    def tocar(self, audio_path):
        if audio_path.exists():
            pygame.mixer.music.load(str(audio_path))
            pygame.mixer.music.play()
        else:
            messagebox.showinfo("Áudio", "Áudio não encontrado.")


class FastEnglishApp:
    def __init__(self, root, palavras):
        self.root = root
        self.root.title("Fast English")
        self.palavras_paths = palavras
        self.indice = 0
        self.audio_player = AudioPlayer()
        self._criar_interface()
        self._atualizar_interface()

    def _criar_interface(self):
        self.top = tk.Frame(self.root)
        self.top.pack()
        tk.Button(self.top, text="< (sair)", command=self.root.quit).pack(side="left")
        self.progresso_label = tk.Label(self.top, text="Progresso...")
        self.progresso_label.pack(side="left", padx=20)

        self.mid = tk.Frame(self.root)
        self.mid.pack()
        self.btn_ant = tk.Button(self.mid, text="<", command=self.anterior)
        self.btn_ant.grid(row=0, column=0)
        self.img_label = tk.Label(self.mid)
        self.img_label.grid(row=0, column=1)
        self.btn_prox = tk.Button(self.mid, text=">", command=self.proximo)
        self.btn_prox.grid(row=0, column=2)

        self.traducao_label = tk.Label(self.root, text="Tradução:")
        self.traducao_label.pack()
        self.tipo_label = tk.Label(self.root, text="Tipo:")
        self.tipo_label.pack()
        self.palavra_label = tk.Label(self.root, text="Palavra:")
        self.palavra_label.pack()
        self.fonetica_label = tk.Label(self.root)
        self.fonetica_label.pack()

        self.btn_audio = tk.Button(self.root, text="🔊 Ouvir", command=self.tocar_audio)
        self.btn_audio.pack(pady=5)

    def _atualizar_interface(self):
        self.palavra = Palavra(self.palavras_paths[self.indice])
        self.progresso_label.config(text=f"{self.indice + 1}/{len(self.palavras_paths)}")

        if self.palavra.imagens:
            img = Image.open(self.palavra.imagens[0]).resize((200, 200))
            photo = ImageTk.PhotoImage(img)
            self.img_label.config(image=photo)
            self.img_label.image = photo

        self.traducao_label.config(text=f"Tradução: {self.palavra.texto_info.get('recorte_1.jpg', '')}")
        self.tipo_label.config(text=f"Tipo: {self.palavra.texto_info.get('recorte_2.jpg', '')}")
        self.palavra_label.config(text=f"Palavra: {self.palavra.texto_info.get('recorte_3.jpg', '')}")

        if self.palavra.fonetica_path.exists():
            fon_img = Image.open(self.palavra.fonetica_path).resize((150, 50))
            fon_photo = ImageTk.PhotoImage(fon_img)
            self.fonetica_label.config(image=fon_photo)
            self.fonetica_label.image = fon_photo

    def tocar_audio(self):
        self.audio_player.tocar(self.palavra.audio_path)

    def proximo(self):
        if self.indice < len(self.palavras_paths) - 1:
            self.indice += 1
            self._atualizar_interface()

    def anterior(self):
        if self.indice > 0:
            self.indice -= 1
            self._atualizar_interface()

if __name__ == "__main__":
    loader = DataLoader()
    palavras = loader.get_all_words(subcategory_name="descrição_da_casa")

    root = tk.Tk()
    app = FastEnglishApp(root, palavras)
    root.mainloop()


invalid command name "132415929921472update_loop"
    while executing
"132415929921472update_loop"
    ("after" script)


In [7]:
root.destroy()

---


In [39]:
import tkinter as tk
import time
from typing import Optional, Callable


class TimerWidget:
    def __init__(self, master: tk.Widget, on_tick: Optional[Callable[[float], None]] = None):
        self.master = master
        self.on_tick = on_tick  # Função chamada a cada atualização com tempo decorrido

        self.running = False
        self.start_time = None
        self.elapsed = 0.0

        self.label = tk.Label(master, text="Tempo: 0.00 s", font=("Arial", 16))
        self.label.pack(pady=10)

        self.play_button = tk.Button(master, text="▶ Play", command=self.start_timer)
        self.play_button.pack(side=tk.LEFT, padx=10)

        self.stop_button = tk.Button(master, text="⏸ Stop", command=self.stop_timer, state=tk.DISABLED)
        self.stop_button.pack(side=tk.RIGHT, padx=10)

        self.update_loop()

    def start_timer(self):
        if not self.running:
            self.running = True
            self.start_time = time.time() - self.elapsed
            self.play_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)

    def stop_timer(self):
        if self.running:
            self.running = False
            self.elapsed = time.time() - self.start_time
            self.play_button.config(state=tk.NORMAL)
            self.stop_button.config(state=tk.DISABLED)

    def reset(self):
        self.running = False
        self.start_time = None
        self.elapsed = 0.0
        self.label.config(text="Tempo: 0.00 s")
        self.play_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)

    def get_time(self) -> float:
        return self.elapsed

    def update_loop(self):
        if self.running:
            self.elapsed = time.time() - self.start_time
            if self.on_tick:
                self.on_tick(self.elapsed)
        self.label.config(text=f"Tempo: {self.elapsed:.2f} s")
        self.master.after(100, self.update_loop)

if __name__ == "__main__":
    root = tk.Tk()
    root.title("Timer Exemplo")

    def mostrar_tempo_em_segundo(t):
        print(f"[INFO] Tempo atual: {t:.2f} segundos")

    timer = TimerWidget(root, on_tick=mostrar_tempo_em_segundo)

    root.mainloop()


In [ ]:
# xuxu e pepino japones

In [107]:
import pandas as pd

df = pd.read_csv("/media/guilherme/ssd_m2_data/py_new_projects/fast_english/database/infos/game_data_hangman.csv")

df.tail()

,datetime,word,hint,won,difficulty,total_attempts,used_attempts,correct_guessed_letters,incorrect_guessed_letters,correct_guesses,incorrect_guesses,time_taken,game_name,clicks_on_guess,category,sub_category
152,2025-05-01T18:50:33,toilet,o banheiro,True,0.33,6,2,"t,o,i,l,e,t","b,a",NaN,NaN,9.002414,hangman,2.0,casa,divisões
153,2025-05-01T18:50:57,larder,a despensa,True,0.00,6,0,"l,a,r,d,e,r",NaN,NaN,NaN,21.006258,hangman,5.0,casa,divisões
154,2025-05-01T18:51:03,loft,o loft,True,0.00,6,0,"l,o,f,t",NaN,NaN,NaN,3.000917,hangman,1.0,casa,divisões
155,2025-05-01T19:01:42,larder,a despensa,True,0.17,6,1,"l,a,r,d,e,r",y,NaN,NaN,25.007209,hangman,5.0,casa,divisões
156,2025-05-01T19:02:05,attic,o sótão,False,1.00,6,6,"a,t,t","d,e,o,n,m,l",NaN,NaN,20.005174,hangman,7.0,casa,divisões


In [108]:
df.groupby("word").agg(
    time_taken_mean = ("time_taken", "mean")
).reset_index().sort_values(by="time_taken_mean", ascending=False)

,word,time_taken_mean
49,|uxurious,131.542339
25,laundry,118.535619
17,drapes,83.030297
10,chandelier,72.541559
22,hallway,60.016492
36,spacious,57.017929
47,window shade,50.520169
26,living room,49.015005
8,candlestick,42.681083
19,flowerpot,41.214230
